In [4]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import cerberus
import subprocess
import re

p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [5]:
major_set = '../swan/isos_sample_gene_90.tsv'
c_annot = 'cerberus_annot_triplets.h5'
gtf = '../cerberus.gtf'
swan_file = '../swan/swan.p'
filt_ab = '../cerberus_filtered_abundance.tsv'
h5 = '../cerberus_annot.h5'

In [3]:
ca = cerberus.read(c_annot)

In [6]:
ca.t_map.head()

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source
0,ENST00000003912.7,2,ENSG00000001461_2,ENSG00000001461_1,1,ENSG00000001461_1,1,ENSG00000001461,NIPAL3,NIPAL3-201,"[1,2,1]","ENSG00000001461[1,2,1]","NIPAL3[1,2,1]",False,False,v40
1,ENST00000008440.9,3,ENSG00000010072_3,ENSG00000010072_1,1,ENSG00000010072_2,2,ENSG00000010072,SPRTN,SPRTN-201,"[1,3,2]","ENSG00000010072[1,3,2]","SPRTN[1,3,2]",False,False,v40
2,ENST00000009105.5,2,ENSG00000008118_2,ENSG00000008118_1,1,ENSG00000008118_1,1,ENSG00000008118,CAMK1G,CAMK1G-201,"[1,2,1]","ENSG00000008118[1,2,1]","CAMK1G[1,2,1]",False,False,v40
3,ENST00000010299.10,2,ENSG00000009780_2,ENSG00000009780_1,1,ENSG00000009780_3,3,ENSG00000009780,FAM76A,FAM76A-201,"[1,2,3]","ENSG00000009780[1,2,3]","FAM76A[1,2,3]",False,False,v40
4,ENST00000011700.10,6,ENSG00000048707_6,ENSG00000048707_5,5,ENSG00000048707_13,13,ENSG00000048707,VPS13D,VPS13D-201,"[5,6,13]","ENSG00000048707[5,6,13]","VPS13D[5,6,13]",False,False,v40


In [9]:
ic = ca.ic[['Name', 'novelty']]

In [20]:
df = ca.t_map.loc[ca.t_map.source=='lapa'].copy(deep=True)

In [21]:
df = df.merge(ca.ic[['Name', 'novelty']], how='left', left_on='ic_id', right_on='Name')
df.rename({'novelty':'ic_novelty'}, axis=1, inplace=True)
df.drop('Name', axis=1, inplace=True)
df = df.merge(ca.tss[['Name', 'novelty']], how='left', left_on='tss_id', right_on='Name')
df.rename({'novelty':'tss_novelty'}, axis=1, inplace=True)
df.drop('Name', axis=1, inplace=True)
df = df.merge(ca.tes[['Name', 'novelty']], how='left', left_on='tes_id', right_on='Name')
df.rename({'novelty':'tes_novelty'}, axis=1, inplace=True)
df.drop('Name', axis=1, inplace=True)


In [22]:
df.head()

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source,ic_novelty,tss_novelty,tes_novelty
0,ENCODEHT000208597,32,ENSG00000228794_32,ENSG00000228794_2,2,ENSG00000228794_7,7,ENSG00000228794,LINC01128,ENCODEHT000208597,"[2,32,7]","ENSG00000228794[2,32,7]","LINC01128[2,32,7]",False,False,lapa,Known,Known,Known
1,ENCODEHT000208597#0,32,ENSG00000228794_32,ENSG00000228794_2,2,ENSG00000228794_8,8,ENSG00000228794,LINC01128,ENCODEHT000208597,"[2,32,8]","ENSG00000228794[2,32,8]","LINC01128[2,32,8]",False,False,lapa,Known,Known,Known
2,ENCODEHT000208597#1,32,ENSG00000228794_32,ENSG00000228794_2,2,ENSG00000228794_8,8,ENSG00000228794,LINC01128,ENCODEHT000208597,"[2,32,8]","ENSG00000228794[2,32,8]","LINC01128[2,32,8]",False,False,lapa,Known,Known,Known
3,ENCODEHT000208597#2,32,ENSG00000228794_32,ENSG00000228794_2,2,ENSG00000228794_19,19,ENSG00000228794,LINC01128,ENCODEHT000208597,"[2,32,19]","ENSG00000228794[2,32,19]","LINC01128[2,32,19]",False,False,lapa,Known,Known,Novel
4,ENCODEHT000208825#0,29,ENSG00000187634_29,ENSG00000187634_10,10,ENSG00000187634_5,5,ENSG00000187634,SAMD11,ENCODEHT000208825,"[10,29,5]","ENSG00000187634[10,29,5]","SAMD11[10,29,5]",False,False,lapa,ISM,Novel,Known


In [24]:
df.to_csv('cerberus_transcript_novelty.tsv', sep='\t', index=False)